# Indexes

Table indexes are data structures that allow fast lookups by an indexed attribute or combination of attributes.

In DataJoint, indexes are created by one of the three mechanisms:

1. Primary key 
2. Foreign key 
3. Explicitly defined indexes

The first two mechanisms are obligatory. Every table has a primary key, which serves as an unique index. Therefore, restrictions by a primary key are very fast. Foreign keys create additional indexes unless a suitable index already exists.

Let's test this principle. Let's create a table with a 10,000 entries and compare lookup times:

In [ ]:
import datajoint as dj
schema = dj.schema('test_indexes')
schema.drop() # drop previous schema definition (if any) and create anew
schema = dj.schema('test_indexes')

Let's say a mouse in the lab has a lab-specific ID but it also has a separate id issued by the animal facility.

In [ ]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id : int  # lab-specific ID
    ---
    tag_id : int  # animal facility ID
    """

In [ ]:
import random
def populate_mice(n=200_000):
    """insert a bunch of mice"""
    Mouse.insert(
        ((random.randint(1,1000_000_000), random.randint(1,1000_000_000)) 
         for i in range(n)), skip_duplicates=True)

In [ ]:
populate_mice()

In [ ]:
Mouse()

In [ ]:
%%timeit -n10 -r10

# efficient! Uses the primary key
(Mouse() & {'mouse_id': random.randint(0, 999_999)}).fetch()

In [ ]:
%%timeit -n10 -r10

# inefficient! Requires a full table scan
(Mouse() & {'tag_id': random.randint(0, 999_999)}).fetch()

The indexed searches are much faster!

To make searches faster on fields other than the primary key or a foreign key, you can add a secondary index explicitly. 

Regular indexes are declared as `index(attr1, ..., attrN)` on a separate line anywhere in the table declration (below the primary key divide). 

Indexes can be declared with unique constraint as  `unique index (attr1, ..., attrN)`.

Let's redeclare the table with a unique index on `tag_id`.

In [ ]:
Mouse.drop()

In [ ]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id : int  # lab-specific ID
    ---
    tag_id : int  # animal facility ID
    unique index (tag_id)
    """

In [ ]:
populate_mice()

In [ ]:
Mouse()

Now both types of searches are equally efficient!

In [ ]:
%%timeit -n10 -r10

#efficient! Uses the primary key
(Mouse() & {'mouse_id': random.randint(0, 999_999)}).fetch()

In [ ]:
%%timeit -n10 -r10

#efficient! Uses the seconary index on tag_id
(Mouse() & {'tag_id': random.randint(0, 999_999)}).fetch()

Let's now imagine that rats in the `Rat` table are identified by the combination of lab the `lab_name` and `rat_id` in each lab:

In [ ]:
@schema
class Rat(dj.Manual):
    definition = """
    lab_name : char(16) 
    rat_id : int unsigned # lab-specific ID
    ---
    date_of_birth = null : date
    """

In [ ]:
def populate_rats():
    lab_names = ("Cajal", "Kandel", "Moser", "Wiesel")
    for date_of_birth in (None, "2019-10-01", "2019-10-02", "2019-10-03", "2019-10-04"):
        Rat.insert((
            (random.choice(lab_names), random.randint(1, 1_000_000), date_of_birth) 
                   for i in range(100_000)), skip_duplicates=True)

In [ ]:
populate_rats()

In [ ]:
Rat()

Note that dispite the fact that `rat_id` is in the index, search by `rat_id` alone are not helped by the index because it is not first in the index. This is similar to search for a word in a dictionary that orders words alphabetically. Searching by the first letters of a word is easy but searching by the last few letters of a word requires scanning the whole dictionary.

In this table, the primary key is a unique index on the combination `(lab_id, rat_id)`. Therefore searches on these attributes or on `lab_id` alone are fast. But this index cannot help searches on `rat_id` alone:

In [ ]:
%%timeit -n2 -r10

# inefficient!  Requires full table scan.
(Rat() & {'rat_id': 300}).fetch()

In [ ]:
%%timeit -n2 -r10

# efficient! Uses the primary key
(Rat() & {'rat_id': 300, 'lab_name': 'Cajal'}).fetch()

In [ ]:
%%timeit -n2 -r10

# inefficient! Requires a full table scan
len(Rat & {'rat_id': 500})

Pattern searches in strings can benefit from an index when the starting characters are specified.

In [ ]:
%%timeit -n2 -r10

# efficient! Uses the primary key
len(Rat & 'lab_name LIKE "Caj%"')

In [ ]:
%%timeit -n2 -r10

# inefficient! requires a full table scan
len(Rat & 'lab_name LIKE "%jal"')

Similarly, searching by the date requires an inefficient full-table scan:

In [ ]:
%%timeit -n2 -r10

len(Rat & 'date_of_birth > "2019-10-02"')

To speed up searches by the `rat_id` and `date_of_birth`, we can explicit indexes to `Rat`:

In [ ]:
Rat.drop()

In [ ]:
@schema
class Rat(dj.Manual):
    definition = """
    lab_name : char(16) 
    rat_id : int unsigned # lab-specific ID
    ---
    date_of_birth = null : date

    index(rat_id)
    index(date_of_birth)
    """

In [ ]:
populate_rats()

In [ ]:
%%timeit -n2 -r10

# efficient!  uses index on rat_id
(Rat() & {'rat_id': 300}).fetch()

In [ ]:
%%timeit -n2 -r10

# efficient! uses index on date_of_birth
len(Rat & 'date_of_birth > "2019-10-02"')

#### Quiz: How many indexes does the table `Rat` have?

In [ ]:
Rat.describe();

#### Answer
